In [19]:
import os, pathlib
import pandas as pd
import warnings

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from import_clean import *
from calculate import *

In [20]:
df_edu = factor_in_age(cleanup_all(import_data("rgn", target_groups=["Highest level of qualification"]), remove_geography=False)["TS067"])
df_edu = prob_calc(df_edu)

areas = df_edu["geography"].tolist()

df_edu
fig, ax = plt.subplots(len(areas)//2, 2, figsize=(5*2,5*len(areas)//2))
dataframes = []
for i, area in enumerate(areas):
    area_data = (
        df_edu.loc[
            df_edu["geography"] == area,
            df_edu.columns.drop(
                ["geography", "Total"]
            )
        ]
        .squeeze()
        .to_frame("Percentage")
        .reset_index(names="Highest Level of Qualification")
    )
    area_data["Highest Level of Qualification"] = (
        area_data["Highest Level of Qualification"].str.split(":")
        .apply(lambda x: x[-1])
    )
    area_data["Percentage"] *= 100
    dataframes.append(area_data)

    sns.barplot(area_data, x="Highest Level of Qualification", y="Percentage", ax=ax[i//2, i%2])
    ax[i//2, i%2].set_xticklabels(list(area_data["Highest Level of Qualification"])[:8], rotation=60, ha="right")
    ax[i//2, i%2].set_title(area)
fig.tight_layout()
areas_to_add = []
for area in areas:
    for i in range(8):
        areas_to_add.append(area)
    

df_edu = pd.concat(dataframes, axis=0)
df_edu["Region"] = areas_to_add
df_edu

UnboundLocalError: local variable 'valid_target_codes' referenced before assignment

In [ ]:
ax = sns.catplot(df_edu, x="Highest Level of Qualification", y="Percentage", hue="Region", kind="bar", ax=ax)
ax.set_xticklabels(list(df_edu["Highest Level of Qualification"])[:8], rotation=60, ha="right")

NameError: name 'df_edu' is not defined